In [138]:
import pandas as pd
import numpy as np

from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from collections import deque


In [139]:
# Import the files
df_train = pd.read_parquet("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/data/train.parquet")
df_test = pd.read_parquet("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/data/final_test.parquet")

In [140]:
# Extract the date feature on different time scales :

def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour

    # creation of a binary varible depicting if day in weekend
    X["is_weekend"] = np.where(X["weekday"] + 1 > 5, 1, 0)

    # Finally we can drop the original columns from the dataframe
    return X

df_train = _encode_dates(df_train)
df_test = _encode_dates(df_test)


In [141]:
df_train.head()

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,log_bike_count,year,month,day,weekday,hour,is_weekend
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000,2020,9,1,1,2,0
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.693147,2020,9,1,1,3,0
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000,2020,9,1,1,4,0
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1.609438,2020,9,1,1,15,0
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2.302585,2020,9,1,1,18,0


In [142]:
df_test.head()

,counter_id,counter_name,site_id,site_name,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,year,month,day,weekday,hour,is_weekend
0,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10 01:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2021,9,10,4,1,0
1,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10 13:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2021,9,10,4,13,0
2,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10 17:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2021,9,10,4,17,0
3,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10 19:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2021,9,10,4,19,0
4,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10 22:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2021,9,10,4,22,0


In [143]:
df_train.describe()

,site_id,bike_count,date,counter_installation_date,latitude,longitude,log_bike_count,year,month,day,weekday,hour,is_weekend
count,4.968270e+05,496827.000000,496827,496827,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000
mean,1.053450e+08,60.191475,2021-03-08 07:25:59.668858,2019-04-04 07:24:35.245911,48.854343,2.345479,3.079917,2020.679846,6.556904,15.458226,2.992172,11.502730,0.283354
min,1.000070e+08,0.000000,2020-09-01 01:00:00,2013-01-18 00:00:00,48.826360,2.265420,0.000000,2020.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,1.000475e+08,5.000000,2020-12-05 22:00:00,2018-11-29 00:00:00,48.840801,2.314440,1.791759,2020.000000,4.000000,8.000000,1.000000,6.000000,0.000000
50%,1.000562e+08,29.000000,2021-03-08 11:00:00,2019-11-06 00:00:00,48.852090,2.353870,3.401197,2021.000000,7.000000,15.000000,3.000000,12.000000,0.000000
75%,1.000563e+08,79.000000,2021-06-09 14:00:00,2019-12-11 00:00:00,48.864610,2.375870,4.382027,2021.000000,9.000000,23.000000,5.000000,18.000000,1.000000
max,3.000147e+08,1302.000000,2021-09-09 23:00:00,2020-11-29 00:00:00,48.891720,2.409690,7.172425,2021.000000,12.000000,31.000000,6.000000,23.000000,1.000000
std,3.210346e+07,87.590566,NaN,NaN,0.018554,0.038026,1.659899,0.466536,3.423834,8.851485,1.995015,6.920936,0.450627


In [144]:
df_test.describe()

,site_id,date,counter_installation_date,latitude,longitude,year,month,day,weekday,hour,is_weekend
count,5.144000e+04,51440,51440,51440.000000,51440.000000,51440.0,51440.000000,51440.000000,51440.000000,51440.000000,51440.000000
mean,1.073050e+08,2021-09-29 13:16:37.908242,2019-05-05 04:44:51.881804,48.854275,2.344642,2021.0,9.466116,15.090921,3.081532,11.491660,0.308865
min,1.000070e+08,2021-09-10 01:00:00,2013-01-18 00:00:00,48.826360,2.265420,2021.0,9.000000,1.000000,0.000000,0.000000,0.000000
25%,1.000475e+08,2021-09-19 18:00:00,2018-11-29 00:00:00,48.839770,2.311790,2021.0,9.000000,10.000000,1.000000,6.000000,0.000000
50%,1.000563e+08,2021-09-29 16:00:00,2019-11-06 00:00:00,48.852090,2.353870,2021.0,9.000000,15.000000,3.000000,11.000000,0.000000
75%,1.000563e+08,2021-10-09 08:00:00,2019-12-12 00:00:00,48.864610,2.375870,2021.0,10.000000,21.000000,5.000000,17.000000,1.000000
max,3.000147e+08,2021-10-18 21:00:00,2020-11-29 00:00:00,48.891720,2.409690,2021.0,10.000000,30.000000,6.000000,23.000000,1.000000
std,3.738839e+07,NaN,NaN,0.018607,0.038257,0.0,0.498855,7.716252,2.042702,6.903709,0.462030


In [145]:
# Adding lag features for the log_bike_count
# We choose lag_1 and lag_24 because lag_1 captures the very short term specificities, while lag_24 captures the daily seasonality.
df_train = df_train.sort_values(by=['counter_id', 'date'])
df_train['lag_1'] = df_train.groupby('counter_id')['log_bike_count'].shift(1)
df_train['lag_24'] = df_train.groupby('counter_id')['log_bike_count'].shift(24)

/var/folders/kg/kfs42kh56l91cr2y6vt0_djr0000gn/T/ipykernel_14767/3182609263.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_train['lag_1'] = df_train.groupby('counter_id')['log_bike_count'].shift(1)
/var/folders/kg/kfs42kh56l91cr2y6vt0_djr0000gn/T/ipykernel_14767/3182609263.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_train['lag_24'] = df_train.groupby('counter_id')['log_bike_count'].shift(24)


In [146]:
# Similarly to the lagged features, we can create rolling means that allows to rely less on noise and add information on average values over different time windows.
# Short term averages (such as a 3-hour window) allows to capture patterns over a few hours, such as the rush hours. on the other hand, daily trends (24_hour window) allows to capture the daily patterns
df_train['rolling_mean_3'] = df_train.groupby('counter_id')['log_bike_count'].transform(lambda x: x.rolling(window=3).mean())
df_train['rolling_mean_24'] = df_train.groupby('counter_id')['log_bike_count'].transform(lambda x: x.rolling(window=24).mean())


/var/folders/kg/kfs42kh56l91cr2y6vt0_djr0000gn/T/ipykernel_14767/3973267024.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_train['rolling_mean_3'] = df_train.groupby('counter_id')['log_bike_count'].transform(lambda x: x.rolling(window=3).mean())
/var/folders/kg/kfs42kh56l91cr2y6vt0_djr0000gn/T/ipykernel_14767/3973267024.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_train['rolling_mean_24'] = df_train.groupby('counter_id')['log_bike_count'].transform(lambda x: x.rolling(window=24).mean())


In [147]:
# Handling Missing Values generated by the lag/rolling features)
lag_rolling_features = ['lag_1', 'lag_24', 'rolling_mean_3', 'rolling_mean_24']

# Fill missing values only in lag and rolling feature columns
df_train[lag_rolling_features] = df_train[lag_rolling_features].fillna(0)

# Verify that missing values have been handled for these features
df_train[lag_rolling_features].isnull().sum()

lag_1              0
lag_24             0
rolling_mean_3     0
rolling_mean_24    0
dtype: int64

We want to include lagged variables and rolling stats to our data set. Therefore, to do this, because the training set and the testing set are sequentially connected, which means that the test df follows immediately the training set in time, we can use the last values of log_bike_count from the traininsg dataset to initialize lagged features for the first rows of the test data set. 
Then, for each test row, the current lagged and rolling features will be used to predict the log bike count and the features will be updated in the subsequent rows

In [148]:
# Sort test data by date
testing_data = df_test.sort_values(by=['counter_id', 'date'])

# Initialize lagged features with the last values from training data
lag_1 = df_train.groupby('counter_id')['log_bike_count'].tail(1).to_dict()
lag_24 = df_train.groupby('counter_id')['log_bike_count'].apply(lambda x: x.tail(24).mean()).to_dict()

rolling_window_3 = {counter_id: deque(df_train[df_train['counter_id'] == counter_id]['log_bike_count'].tail(3), maxlen=3)
                    for counter_id in df_train['counter_id'].unique()}

rolling_window_24 = {counter_id: deque(df_train[df_train['counter_id'] == counter_id]['log_bike_count'].tail(24), maxlen=24)
                     for counter_id in df_train['counter_id'].unique()}

# Placeholder for predictions
testing_data['log_bike_count_pred'] = np.nan


X_train = df_train.drop(columns=["bike_count", "log_bike_count"])
y_train = df_train["log_bike_count"]

X_test = df_test.copy()

/var/folders/kg/kfs42kh56l91cr2y6vt0_djr0000gn/T/ipykernel_14767/4172974288.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  lag_1 = df_train.groupby('counter_id')['log_bike_count'].tail(1).to_dict()
/var/folders/kg/kfs42kh56l91cr2y6vt0_djr0000gn/T/ipykernel_14767/4172974288.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  lag_24 = df_train.groupby('counter_id')['log_bike_count'].apply(lambda x: x.tail(24).mean()).to_dict()


'\nmapper = DataFrameMapper(\n    [\n        (["latitude", "longitude"], StandardScaler()),  # Normalize numerical features\n        (["counter_id", "site_id"], [SimpleImputer(strategy="most_frequent"), OneHotEncoder()]),  # Encode categorical features\n        (["hour", "day", "month", "year", "is_weekend"], OneHotEncoder()),  # Encode datetime features\n        (["lag_1", "lag_24", "rolling_mean_3", "rolling_mean_24"], StandardScaler()),  # Scale lag/rolling stats\n    ],\n    df_out=True,  # Return pandas DataFrame\n    default=None,  # Default to skipping unknown columns\n)\n'

In [149]:
# Define preprocessing steps for numerical and categorical columns
numerical_features = ['latitude', 'longitude', 'year', 'month', 'day', 'weekday', 'hour', 'lag_1', 'lag_24', 'rolling_mean_3', 'rolling_mean_24']
categorical_features = ['counter_id', 'site_id', 'is_weekend']

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),  # Scale numerical features
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),  # Encode categorical features
    ]
)

# Full pipeline with a model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', GradientBoostingRegressor())
])

In [170]:

# Fit Pipeline to Training Data
pipeline.fit(X_train, y_train)

# Make Predictions on Test Data
# y_predictions = pipeline.predict(X_test)

# Columns expected by the pipeline
expected_features = ['latitude', 'longitude', 'year', 'month', 'day', 'weekday', 'hour',
                     'lag_1', 'lag_24', 'rolling_mean_3', 'rolling_mean_24',
                     'counter_id', 'site_id', 'is_weekend']

y_predictions = []  # Initialize an empty list to store predictions

# Sequentially predict for each row in the test dataset
for idx, row in testing_data.iterrows():
    counter_id = row['counter_id']

    # Set lag and rolling features for the current row
    row['lag_1'] = lag_1.get(counter_id, 0)
    row['lag_24'] = lag_24.get(counter_id, 0)
    # Ensure rolling windows contain only numeric scalars
    rolling_3_data = [float(x[0]) if isinstance(x, np.ndarray) else float(x) for x in rolling_window_3.get(counter_id, deque([0], maxlen=3))]
    rolling_24_data = [float(x[0]) if isinstance(x, np.ndarray) else float(x) for x in rolling_window_24.get(counter_id, deque([0], maxlen=24))]

    row['rolling_mean_3'] = np.mean(rolling_3_data)
    row['rolling_mean_24'] = np.mean(rolling_24_data)

    # Align row with expected features
    row = row.reindex(expected_features, fill_value=0)

    # Predict log_bike_count for the current row using the trained model
    row_features = row[expected_features].to_frame().T.reindex(columns=numerical_features + categorical_features, fill_value=0)

    # Preprocess the row using the pipeline's preprocessor
    row_preprocessed = pipeline['preprocessor'].transform(row_features)

    # Make prediction using the preprocessed row
    prediction = pipeline['model'].predict(row_preprocessed)

    # Store the prediction
    row['log_bike_count_pred'] = prediction

    # Append the prediction to the list
    y_predictions.append(prediction[0])  # Ensure prediction is a scalar

    # Update lag and rolling windows
    lag_1[counter_id] = prediction[0]  # Ensure it's a scalar
    rolling_window_3[counter_id].append(float(prediction[0]))  # Append only the scalar prediction
    rolling_window_24[counter_id].append(float(prediction[0]))


In [171]:
print(y_predictions)

[3.0238202941926327, 3.0809128856040866, 3.235296792269057, 3.777833689780666, 3.9606687726658785, 4.274581208239533, 4.349073435944799, 3.9358462776622822, 3.932548898817649, 3.9520857727401153, 3.894170692508763, 3.894170692508763, 3.8710904101332066, 3.8710904101332066, 3.941755981629972, 3.941755981629972, 3.885217928795751, 3.548012482427773, 3.311226038498309, 3.1569795792851356, 2.955026145287921, 2.776421296994256, 2.727825123150794, 2.8267289508986377, 2.7593534835030242, 2.8051145978604963, 3.0783702300670472, 3.429506137635837, 3.5541901500227033, 3.6928599609277795, 4.00298165105138, 3.7188848499501916, 3.742114905357599, 3.767929716290291, 3.732087024575766, 3.732087024575766, 3.732087024575766, 3.723917617774497, 3.7529698607835473, 3.761139267584816, 3.684984593445122, 3.3643722288197346, 3.1492080539061797, 2.993742406276699, 2.7686497716153, 2.7200535977718383, 2.663213459823042, 2.694860088567993, 2.6369745275169105, 2.6827356418743826, 2.9559912740809335, 3.284622872

In [173]:
pd.DataFrame(y_predictions, columns=["log_bike_count"]).reset_index().rename(
    columns={"index": "Id"}
).to_csv("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/predictions.csv", index=False)